# Problem Set #7
### MACS 30100, Dr. Evans
March 5, 2018  
Mengchen Shi

In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, mean_squared_error

from pylab import rcParams
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [2]:
df = pd.read_csv("Auto.csv", na_values='?')

In [3]:
df.columns=['mpg','cyl','dspl','hpwr','wgt','accl','yr','orgn','name']

In [4]:
df.dropna(inplace=True)

In [5]:
df.head()

,mpg,cyl,dspl,hpwr,wgt,accl,yr,orgn,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [6]:
df.isnull().sum()

mpg     0
cyl     0
dspl    0
hpwr    0
wgt     0
accl    0
yr      0
orgn    0
name    0
dtype: int64

In [7]:
df['mpg_high'] = df['mpg'] >= df['mpg'].median()

In [8]:
df['mpg_high'] = pd.get_dummies(df['mpg_high'], drop_first=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 396
Data columns (total 10 columns):
mpg         392 non-null float64
cyl         392 non-null int64
dspl        392 non-null float64
hpwr        392 non-null float64
wgt         392 non-null int64
accl        392 non-null float64
yr          392 non-null int64
orgn        392 non-null int64
name        392 non-null object
mpg_high    392 non-null uint8
dtypes: float64(4), int64(4), object(1), uint8(1)
memory usage: 31.0+ KB


In [10]:
df['hpwr'] = df['hpwr'].convert_objects(convert_numeric = True)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


## (a)

In [11]:
X = df[['cyl','dspl','hpwr','wgt','accl','yr','orgn']].dropna().values
y = df['mpg_high'].dropna().values

In [12]:
kf_log = KFold(n_splits=4, shuffle=True, random_state=15)

In [13]:
LogReg = LogisticRegression(fit_intercept=True)
#LogReg.fit(X_train, y_train)

In [14]:
k_ind = int(0)
log_err_0 = np.zeros(4)
log_err_1 = np.zeros(4)
log_MSE_VEC = np.zeros(4)

for train_index, test_index in kf_log.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    
    error = y_test != y_pred
    #print(error)
    
    log_err_0[k_ind] = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    log_err_1[k_ind] = ((y_pred == 1) * error).sum() / (y_pred == 1).sum() 
    #print(log_err_0[k_ind])
    #print(((y_test == 0) * error).sum() / (y_test == 0).sum() )
    
    log_MSE_VEC[k_ind] = error.mean()
    
    print('\n',classification_report(y_test, y_pred, digits=3))
    print('error rate (category 0) is', log_err_0[k_ind], '\n'
          'error rate (category 1) is', log_err_1[k_ind])
    print('The MSE of the model is', log_MSE_VEC[k_ind])
    print()
    k_ind += 1
    
print('\n k-Fold Reults:')
print('The average error rate(category 0) is', log_err_0.mean())
print('The average error rate(category 1) is', log_err_1.mean())
print('The average MSE of the model is', log_MSE_VEC.mean())


              precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

error rate (category 0) is 0.057692307692307696 
error rate (category 1) is 0.13043478260869565
The MSE of the model is 0.09183673469387756


              precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

error rate (category 0) is 0.12244897959183673 
error rate (category 1) is 0.08163265306122448
The MSE of the model is 0.10204081632653061


              precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0.867     0.867        98

error rate (category 0) is 0.15217391304347827 
error rate (category

## (b)

In [15]:
clf = RandomForestClassifier(n_estimators=20, random_state=25,bootstrap=True, max_features=2, oob_score=True)
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [16]:
oob_pred = clf.oob_decision_function_.T[1]
MSE_rf = pd.DataFrame({'ypred': oob_pred, 'y': y})
MSE_rf['ypred'] = MSE_rf['ypred'].apply(lambda x: 1 if x>=0.5 else 0)

err_rates_0 = mean_squared_error(MSE_rf['y'].where(MSE_rf.ypred<0.5).dropna(), MSE_rf['ypred'].where(MSE_rf.ypred<0.5).dropna())
err_rates_1 = mean_squared_error(MSE_rf['y'].where(MSE_rf.ypred>=0.5).dropna(), MSE_rf['ypred'].where(MSE_rf.ypred>=0.5).dropna())

MSE = mean_squared_error(MSE_rf['y'], MSE_rf['ypred'])
print('The MSE of the random forest model is: ', MSE)

print('The average error rate(category 0) = ', err_rates_0)
print('The average error rate(category 1) = ', err_rates_1)

The MSE of the random forest model is:  0.07142857142857142
The average error rate(category 0) =  0.05789473684210526
The average error rate(category 1) =  0.08415841584158416


In [17]:
print(classification_report(MSE_rf['y'], MSE_rf['ypred']))

             precision    recall  f1-score   support

          0       0.94      0.91      0.93       196
          1       0.92      0.94      0.93       196

avg / total       0.93      0.93      0.93       392



## (c)

In [18]:
kf_log = KFold(n_splits=4, shuffle=True, random_state=15)

In [20]:
k_ind = int(0)
svm_err_0 = np.zeros(4)
svm_err_1 = np.zeros(4)
svm_MSE_VEC = np.zeros(4)

for train_index, test_index in kf_log.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    svc_rbf = svm.SVC(kernel='rbf', C=1, gamma=0.2)
    svc_rbf.fit(X_train, y_train)
    y_pred_svc = svc_rbf.predict(X_test)
    
    #error = y_test != y_pred
    
    pred_wrong_rbf = y_test != y_pred_svc
    
    MSE_rbf = pred_wrong_rbf.mean()
    print('RBF MSE for test set is', MSE_rbf)
    
    svm_err_0[k_ind] = (((y_pred_svc == 0) * (pred_wrong_rbf)).sum() / (y_pred_svc == 0).sum())
    print('error rate (category 0) is', svm_err_0[k_ind])
    
    svm_err_1[k_ind] = (((y_pred_svc == 1) * (pred_wrong_rbf)).sum() / (y_pred_svc == 1).sum())
    print('error rate (category 1) is', svm_err_1[k_ind])
    
    svm_MSE_VEC[k_ind] = pred_wrong_rbf.mean()
    print(classification_report(y_test, y_pred_svc), '\n','\n')
    
    k_ind += 1
    
print('\n k-Fold Reults:')
print('The average error rate(category 0) is', svm_err_0[~np.isnan(svm_err_0)].mean())
print('The average error rate(category 1) is', svm_err_1[~np.isnan(svm_err_1)].mean())
print('The average MSE of the model is', svm_MSE_VEC.mean())

RBF MSE for test set is 0.5408163265306123
error rate (category 0) is 0.0
error rate (category 1) is 0.5520833333333334
             precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98
 
 

RBF MSE for test set is 0.5204081632653061
error rate (category 0) is 0.5204081632653061
error rate (category 1) is nan
             precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98
 
 

RBF MSE for test set is 0.5204081632653061
error rate (category 0) is 0.53125
error rate (category 1) is 0.0
             precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## (d)

(a):  
 k-Fold Reults:
The average error rate(category 0) is 0.09470670705864986  
The average error rate(category 1) is 0.11822664912727025  
The average MSE of the model is 0.10714285714285715  


(b):  
The average error rate(category 0) =  0.05789473684210526  
The average error rate(category 1) =  0.08415841584158416  
The MSE of the random forest model is:  0.07142857142857142  

(c):
 k-Fold Reults:  
The average error rate(category 0) is 0.38119411070879966  
The average error rate(category 1) is 0.1840277777777778  
The average MSE of the model is 0.5076530612244898  

According to the results, the random forest model has the lowest error rates and lowest MSE. Therefore, the random forest model is the best predictor of mpg_high. The SVM has the worst performance, and the logistic model is in between. 